In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [2]:
#@title DataLoad [gdown]
import gdown, zipfile, os

file_id = '1OvQwfbLsAYN4GPENpLGkW87fpeBqTr9Z'
output = 'file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'cloud'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(output, 'r') as z:
  z.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1OvQwfbLsAYN4GPENpLGkW87fpeBqTr9Z
From (redirected): https://drive.google.com/uc?id=1OvQwfbLsAYN4GPENpLGkW87fpeBqTr9Z&confirm=t&uuid=c41b4f00-ea1e-47b8-8b1f-4c4371648238
To: /content/file.zip
100%|██████████| 97.7M/97.7M [00:01<00:00, 77.3MB/s]


In [3]:
#@title DataLoad
from sklearn.model_selection import train_test_split
import numpy as np
def load_data(base_dir='cloud/data/train'):
  images = []
  labels = []
  class_names = []

  for class_name in os.listdir(base_dir):
    class_name_path = os.path.join(base_dir, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        images.append(image_path)
        labels.append(class_name)

  return np.array(images), np.array(labels)

images, labels = load_data()
print(images.shape, labels.shape)
print(np.unique(labels))

(2323,) (2323,)
['Ac' 'As' 'Cb' 'Cc' 'Ci' 'Cs' 'Ct' 'Cu' 'Ns' 'Sc' 'St']


In [4]:
from tensorflow.keras.utils import Sequence
import cv2
import matplotlib.pyplot as plt


class cloudAugmentation(Sequence):
  def __init__(self, images, labels, batch_size=32, augmentor=None,
               target_size=(224, 224), save_dir=None):
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.augmentor = augmentor
    self.target_size = target_size
    self.save_dir = save_dir

  def __len__(self):
    return int(np.ceil(self.images.shape[0] / self.batch_size))

  def __getitem__(self, index):
    start = index * self.batch_size
    stop = (index + 1) * self.batch_size
    batch_images_path = self.images[start:stop]
    batch_labels = self.labels[start:stop]

    batch_images = []

    for i, (image_path, label) in enumerate(zip(batch_images_path, batch_labels)):
      image = cv2.imread(image_path)
      # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      # 데이터 증강
      if self.augmentor is not None:
        image = self.augmentor(image=image)['image']

      # 리사이즈
      image = cv2.resize(image, self.target_size)

      # 저장
      if self.save_dir is not None:
        os.makedirs(self.save_dir, exist_ok=True)

        label_dir = os.path.join(self.save_dir, label)
        os.makedirs(label_dir, exist_ok=True)

        aug_image_name = f'aug_batch{index}_{i}.jpg'
        aug_image_path = os.path.join(label_dir, aug_image_name)
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(aug_image_path, image)

        batch_images.append(image)

    return np.array(batch_images), batch_labels

In [5]:
#@title image augment 적용
import albumentations as A

augmentor = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.1, p=1),
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=0.3),
])

save_dir = 'cloud/data_aug'

cloud_aug = cloudAugmentation(images, labels, augmentor=augmentor, save_dir=save_dir)

for i in range(len(cloud_aug)):
  batch_images, batch_labels = cloud_aug[i]
  print(batch_images.shape, batch_labels.shape) # 근데 이거 약간 이상한데?

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3

In [6]:
ls cloud

data/  data_aug/


In [7]:
#@title AugDataLoad
from sklearn.model_selection import train_test_split
import numpy as np
def load_data(base_dir='cloud/data_aug'):
  images = []
  labels = []
  class_names = []

  for class_name in os.listdir(base_dir):
    class_name_path = os.path.join(base_dir, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        images.append(image_path)
        labels.append(class_name)

  return np.array(images), np.array(labels)

images, labels = load_data()
print(images.shape, labels.shape)
print(np.unique(labels)) # 그리고 aug 햇는데 크기가 왜 똑같지?

(2323,) (2323,)
['Ac' 'As' 'Cb' 'Cc' 'Ci' 'Cs' 'Ct' 'Cu' 'Ns' 'Sc' 'St']


In [8]:
#@title train_validation data split
from sklearn.model_selection  import train_test_split

train_images_path ,val_images_path, train_labels, val_labels=train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
train_labels = np.array(train_labels, dtype='int32')
print(np.unique(train_labels))
print(train_labels.dtype)

val_labels = label_encoder.transform(val_labels)
val_labels = np.array(val_labels, dtype='int32')
print(np.unique(val_labels))
print(val_labels.dtype)

[ 0  1  2  3  4  5  6  7  8  9 10]
int32
[ 0  1  2  3  4  5  6  7  8  9 10]
int32


In [10]:
# 분류층을 제외하고 모델 로드
# - input_shape=(224, 224, 3) (기본값)
# - include_top=True (기본값) 분류층 포함여부
# - weights='imagenet' (기본값) imagenet 1000개클래스를 학습한 가중치
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout,Flatten
from tensorflow.keras.models import Model

base_model = VGG16(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(11, activation='softmax')(x)  # 분류클래스 (0~10 : 11개)

model = Model(inputs=base_model.input, outputs=output)

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 64, 64, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 15,242,059 (58.14 MB)

 Trainable params: 15,242,059 (58.14 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

In [12]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

image_size = (64, 64)  # 모델 입력 크기에 맞게 조정
train_images= np.array([img_to_array(load_img(img_path, target_size=image_size)) for img_path in train_images_path])
train_images = train_images / 255.0  # 정규화 (0-1 범위)

val_images = np.array([img_to_array(load_img(img_path, target_size=image_size)) for img_path in val_images_path])
val_images= val_images / 255.0

##### **problem**
- label에 대해 label_encoding을 진행하고 train과 validation label을 나누었음
- 그리고 model에 전달해서 fit하는 과정을 거침
- 그런데 str64, str99 같은 error가 발생함
- train_labels.dtype을 찍어보니 **'<U2'** 로 출력됨
  - U2 : 유니코드 문자열 데이터 타입
- 그래서 split후 다시 int로 변환해주는 작업을 하니 str64, str99와 같은 오류 제거 완료

In [13]:
print("Train images dtype:", train_images.dtype)
print("Validation images dtype:", val_images.dtype)
print("Train labels dtype:", train_labels.dtype)
print("Validation labels dtype:", val_labels.dtype)


Train images dtype: float32
Validation images dtype: float32
Train labels dtype: int32
Validation labels dtype: int32


In [14]:
model.fit(train_images, train_labels,
              epochs=30,
              batch_size=32,
              validation_data=(val_images, val_labels))

Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 26s 224ms/step - accuracy: 0.1053 - loss: 2.7923 - val_accuracy: 0.1527 - val_loss: 2.3546
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.1152 - loss: 2.3962 - val_accuracy: 0.1118 - val_loss: 2.3851
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.1197 - loss: 2.3895 - val_accuracy: 0.1527 - val_loss: 2.3776
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.1305 - loss: 2.3813 - val_accuracy: 0.1527 - val_loss: 2.3680
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.1289 - loss: 2.3751 - val_accuracy: 0.1527 - val_loss: 2.3621
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.1326 - loss: 2.3727 - val_accuracy: 0.1527 - val_loss: 2.3591
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.1384 - loss: 2.3695 - val_accuracy: 0.1527 - val_loss: 2.3567
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.1379 - loss: 2.3644 - val_accuracy: 0.1527 -

KeyboardInterrupt: 